#### Import the relevant libraries

In [73]:
from PIL import Image
from PIL import ImageFilter
import numpy as np
import copy
import time

#### Read and convert to greyscale

Open the image and convert it to greyscale since we are asked to convolve on greyscale images

In [74]:
def read_image(x):
    x = Image.open('test-images/' + x).convert("L")
    x = np.array(x)
    return x

In [75]:
pixels = read_image('music1.png')
pixels.shape

(231, 1274)

#### Custom kernels

In [76]:
def check_range(x): return (np.min(x), np.max(x))

In [77]:
check_range(pixels)

(0, 255)

**Some test kernels**

- a = identity
- b = box blur
- c = horizontal derivative
- d = gaussian
- e = sharpening
- f = derivative of gaussian

In [78]:
a = np.zeros((3,3))
a[1,1] = 1
a

array([[0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.]])

In [79]:
b = np.ones((3,3))
b

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [80]:
c = np.zeros((3,3))
c[1,0], c[1,2] = -1, 1
c

array([[ 0.,  0.,  0.],
       [-1.,  0.,  1.],
       [ 0.,  0.,  0.]])

In [81]:
d = np.array([[0.003,0.013,0.022,0.013,0.003],
             [0.013,0.059,0.097,0.059,0.013],
             [0.022,0.097,0.159,0.097,0.022],
             [0.013,0.059,0.097,0.059,0.013],
             [0.003,0.013,0.022,0.013,0.003]])
d

array([[0.003, 0.013, 0.022, 0.013, 0.003],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.022, 0.097, 0.159, 0.097, 0.022],
       [0.013, 0.059, 0.097, 0.059, 0.013],
       [0.003, 0.013, 0.022, 0.013, 0.003]])

In [82]:
# vary the value of alpha
e = np.zeros((5,5))
e[1:4,1:4] = a
alpha = 0.8 + 1
e = e * alpha - d
e

array([[-0.003, -0.013, -0.022, -0.013, -0.003],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.022, -0.097,  1.641, -0.097, -0.022],
       [-0.013, -0.059, -0.097, -0.059, -0.013],
       [-0.003, -0.013, -0.022, -0.013, -0.003]])

In [83]:
f = np.zeros((5,5))
f[1:4,1:4] = c
f = f * d
f

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.097,  0.   ,  0.097,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ]])

#### Padding the image

In [84]:
def pad_image(x, kernel):
    """
    Create a new numpy array with all values 255 and then add the image in between
    i/p: image, kernel
    o/p: padded_kernelx
    """
    to_add = kernel.shape[0] - 1
    padded_image = np.full((x.shape[0] + to_add, x.shape[1] + to_add), 255)
    t = to_add // 2
    padded_image[t: padded_image.shape[0] - t, t: padded_image.shape[1] - t] = x
    return padded_image

In [85]:
p = pad_image(pixels, a)
p.shape

(233, 1276)

In [86]:
assert p.shape == (pixels.shape[0] + a.shape[0] - 1, pixels.shape[1] + a.shape[0] - 1)

In [87]:
def display_np(x):
    """
    Display a numpy array as an image
    """
    result = Image.fromarray(x.astype('uint8'))
    result.show()

In [88]:
# display_np(p)

#### convolution

In [89]:
def conv2d(image, kernel, div = 1, clip = True):
    """
    Applies 2d convolution on a 2d image. Also works with rectangular kernels
    """

    # number of rows and columns of the kernel
    r = kernel.shape[0]
    c = kernel.shape[1]

    # initialize a canvas for the output with 255s. We will fill values in this
    output = np.full(image.shape, 255)
    for i in range(image.shape[0] - r - 1):
        for j in range(image.shape[1] - c - 1):
            output[i][j] = np.sum(kernel * image[i:i+r, j:j+c]) / div
    if clip: np.clip(output, 0, 255, out = output)
    return output
#     display_np(output)

In [90]:
# conv2d(p, a)
# conv2d(p, b, 9)
# conv2d(p, c)
# conv2d(p, d)
# conv2d(p, e)
# conv2d(p, f)

In [91]:
sobel = np.array([[-1, 0, 1],
                 [-2, 0, 2],
                 [-1, 0, 1]])

In [92]:
# conv2d(p, sobel)

#### Template matching

In [93]:
%ls test-images/

Invalid switch - "".


In [94]:
tmp1 = read_image('template1.png')
tmp1.shape

(11, 17)

In [131]:
# display_np(tmp1)

In [132]:
op = conv2d(p, tmp1, 1, False) + conv2d(255 - p, 255 - tmp1, 1, False)
op.shape

(233, 1276)

In [133]:
check_range(op)

(510, 11497603)

In [134]:
temp_op = op[op > 10000000]

In [159]:
# m = np.min(op)
idxs1 = np.array([])
idxs2 = np.array([])
for t in temp_op:
    t_idxs = np.where(op == t)
    idxs1 = np.append(idxs1 ,t_idxs[0])
    idxs2 = np.append(idxs2 ,t_idxs[1])

In [161]:
len(idxs1)

484

In [160]:
assert len(idxs1) == len(idxs1)

In [162]:
check_range(idxs1), check_range(idxs2)

((37.0, 195.0), (108.0, 1085.0))

In [163]:
canvas = np.full(p.shape, 255)

In [164]:
check_range(canvas)

(255, 255)

In [169]:
for r,c in zip(idxs1, idxs2):
    r, c = int(r), int(c)
    if r+tmp1.shape[0] < canvas.shape[0] and c+tmp1.shape[1] < canvas.shape[1]:
#         display_np(p[r:r+tmp1.shape[0], c:c+tmp1.shape[1]])
#         break
        canvas[r:r+tmp1.shape[0], c:c+tmp1.shape[1]] = p[r:r+tmp1.shape[0], c:c+tmp1.shape[1]]
#         display_np(canvas)
#         print(check_range(canvas))
#         time.sleep(5)

In [170]:
display_np(canvas)

#### 1d convolution

In [ ]:
def conv1d(image, kernel, div = 1):
    """
    Performs 1d convolution on an image
    """
    
    im = copy.deepcopy(image.flatten())
    k = copy.deepcopy(kernel.flatten())

    size = len(k)
    to_remove = len(k) - 1
    
    output = np.full((im.shape), 255)
    for i in range(len(im) - to_remove):
        output[i] = np.sum(k * im[i:i+size]) / div
    return output

In [ ]:
h1 = np.array([1,2,1])
h2 = np.array([-1,0,1])

In [ ]:
tmp = conv1d(p, h1)

In [ ]:
result = conv1d(tmp, h2)

In [ ]:
check_range(result)

In [ ]:
np.clip(result, 0, 255, out = result)
check_range(result)

In [ ]:
result = result.reshape(p.shape)
result.shape

In [ ]:
# display_np(result)